In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
# specify substep parameters for interactive run
# this cell will be replaced during job run with the parameters from json within params subfolder
substep_params={   
    "min_score" : 0.05
}

In [ ]:
# load pipeline and step parameters - do not edit
from sinara.substep import get_pipeline_params, get_step_params
pipeline_params = get_pipeline_params(pprint=True)
step_params = get_step_params(pprint=True)

In [ ]:
# define substep interface
from sinara.substep import NotebookSubstep, ENV_NAME, PIPELINE_NAME, ZONE_NAME, STEP_NAME, RUN_ID, ENTITY_NAME, ENTITY_PATH, SUBSTEP_NAME

substep = NotebookSubstep(pipeline_params, step_params, substep_params)

substep.interface(
    inputs =    
    [ 
      { STEP_NAME: "data_prep", ENTITY_NAME: "coco_test_dataset"}, # dataset for detector testing from data_prep step
      { STEP_NAME: "model_pack", ENTITY_NAME: "bento_service"}  # stored BentoService from model_pack
    ],
    tmp_outputs =
    [
        { ENTITY_NAME: "coco_test_dataset" },  # ground-true test dataset files for use in next substep
        { ENTITY_NAME: "coco_inference_result_dataset" } # predicted test dataset files for use in next substep
    ],
)

substep.print_interface_info()

substep.exit_in_visualize_mode()

In [ ]:
# run spark
from sinara.spark import SinaraSpark

spark = SinaraSpark.run_session(0)
SinaraSpark.ui_url()

### Loading REST bento_service 

#### Get and load bentoservice

In [ ]:
import os.path as osp

In [ ]:
from sinara.bentoml import load_bentoservice

# read trained model
print('read trained model')
inputs_model_pack = substep.inputs(step_name = "model_pack")
bento_service = load_bentoservice(inputs_model_pack.bento_service)

### Loading test datasets (from step data_prep)

In [ ]:
from sinara.archive import SinaraArchive
import json

# copy test data and dataset_config from previos step (data_prep) to tmp
inputs_data_prep = substep.inputs(step_name = "data_prep")
tmp_outputs = substep.tmp_outputs()

archive = SinaraArchive(spark)
archive.unpack_files_from_store_to_tmp(store_path=inputs_data_prep.coco_test_dataset, tmp_entity_dir=tmp_outputs.coco_test_dataset)

# reading ground-true test dataset markup 
with open(osp.join(tmp_outputs.coco_test_dataset, "test_coco_annotations.json")) as f_id:
    test_coco_dataset = json.load(f_id)
    
print('Annotations count in test_coco_dataset:', len(test_coco_dataset['annotations']))
print('Keys count in test_coco_dataset:', list(test_coco_dataset))

### Inference by REST API bentoservice

#### Inference on a test dataset

##### Inference detector on a test dataset and prepere eval_coco_dataset.json for eval step

In [ ]:
from tqdm import tqdm
import cv2

from utils.coco import convert_inference_results_to_coco
from utils.coco import dump_coco_file

eval_coco_dataset = dict(test_coco_dataset, **{"annotations" : []})
for image in tqdm(test_coco_dataset['images'], desc='process predict'):
    file_name = osp.join(tmp_outputs.coco_test_dataset, image['file_name'])
    img = cv2.imread(file_name)
    out_coco = bento_service.predict(file_name)
    
    out_coco["file_name"] = image["file_name"]
    out_coco["image_id"] = image["id"]    
    out_coco_annotations = []
    for out_coco_ann in  out_coco["annotations"]:
        out_coco_ann.update({"image_id": image["id"]})
        out_coco_annotations.append(out_coco_ann)
        
    eval_coco_dataset["annotations"] += out_coco_annotations
        
last_image_file_name = file_name
        
print('test_coco_dataset annotations amount:', len(test_coco_dataset['annotations']))
print('eval_coco_dataset annotations amount:', len(eval_coco_dataset['annotations']))

eval_coco_dataset['info']['iouType'] = 'bbox'
for ann in eval_coco_dataset['annotations']:
    ann['area'] = ann['bbox'][2] * ann['bbox'][3]

dump_coco_file(osp.join(tmp_outputs.coco_inference_result_dataset, 'eval_coco_dataset.json'), eval_coco_dataset)
print(f"eval_coco_torch = {osp.join(tmp_outputs.coco_inference_result_dataset, 'eval_coco_dataset.json')}")

### Preview of object detection on an image

#### Visualize last predict to last image from coco_test_dataset

In [ ]:
from utils.coco import preview_coco_file

In [ ]:
preview_coco_file(eval_coco_dataset, img_folder=tmp_outputs.coco_test_dataset, count=2)

In [ ]:
# stop spark
SinaraSpark.stop_session()